In [200]:
import numpy as np;
from queue import *
import time

In [207]:
class Stopwatch:
    starttime = 0
    
    def start():
        Stopwatch.starttime = time.time()
        
    def check():
        return time.time() - Stopwatch.starttime

In [57]:
def build_table(l):
    n = len(l)
    
    return([[- i * j if (i < 0 or j < 0) else -1 if (i == j) else (l[(n - i + j) % n] + i) % n for j in range(-1, n)] for i in range(-1, n)])

In [175]:
def print_table(t):
    def find_width(n):
        ans = 0
        if (n < 0):
            ans = 1
            n = -n
        if (n == 0):
            return 1
        while (n > 0):
            ans += 1
            n //= 10
        return ans

    m = len(t)
    n = len(t[0])
    mini = 0
    maxi = 0
    
    for line in t:
        for elem in line:
            if (maxi < elem):
                maxi = elem
            if (elem < mini):
                mini = elem
    
    maxwidth = max(find_width(mini), find_width(maxi))
    
    for line in t:
        for elem in line:
            width = find_width(elem)
            #print(elem, width)
            #for i in range(maxwidth + 1 - width):
            #    print(' ', end = '')
            #print(elem, end = '')
            print(''.join([" " for i in range(maxwidth - width)]), elem, end = '')
        print('')
    #print(t)

In [129]:
def check_table(t):
    n = len(t) - 1
    ans = True
    
    for i in range(-1, n):
        for j in range(-1, n):
            for k in range(-1, n):
                a = t[t[i + 1][j + 1] + 1][k + 1]
                b = t[i + 1][t[j + 1][k + 1] + 1]
                if (a != b):
                    print('alarm', i, j, k, a, b)
                    ans = False
                    return ans
    return ans

In [202]:
def generate_row_recursive(n, p, m, row, free_nums, ans, depth):
    next_pos = 0
    for i in range(1, n):
        if (row[i] == 0):
            next_pos = i
            break
    if (next_pos == 0):
        ans.append(list(row))
        print(list(row))
        return
    
    for i in range(1, n):
        if (i == next_pos):
            continue
        #print('prep: ' + str((next_pos, i)))
        if (free_nums[i] == 0):
            q = Queue()
            changes = []
            q.put((next_pos, i))
            #print("Herer")
            res = insert_recursive(n, p, m, row, free_nums, q, changes)
            #print("Herer2")
            if (res):
                if (depth == 0):
                    #print("progress: pos = ", next_pos, ", num = ", i)
                    time = Stopwatch.check()
                    print("progress:", (i * 10000 // n) / 100, "%, time =", time)
                generate_row_recursive(n, p, m, row, free_nums, ans, depth + 1)
                clean_row(n, row, free_nums, changes)
    return

In [156]:
def clean_row(n, row, free_nums, changes):
    for pos in changes:
        if (row[pos] != 0):
            free_nums[row[pos]] = 0
        row[pos] = 0

In [189]:
def check_n_put(elem, q, row, free_nums):
    (pos, num) = elem
    n = len(row)
    
    if (pos < 1 or n - 1 < pos or num < 1 or n - 1 < num):
        return False
    
    if (row[pos] == num):
        return True
    
    if (row[pos] != 0 or free_nums[num] != 0):
        return False
    
    q.put((pos, num))
    return True

In [185]:
def insert_recursive(n, p, m, row, free_nums, q, changes):
    if (q.empty()):
        return True
    
    (pos, num) = q.get()
    if (pos < 1 or n - 1 < pos or num < 1 or n - 1 < num):
        return False
    
    if (row[pos] == num):
        return insert_recursive(n, p, m, row, free_nums, q, changes)
    
    if (row[pos] != 0 or free_nums[num] != 0):
        return False
    
    free_nums[num] = pos
    row[pos] = num
    changes.append(pos)
    
    flag = True
    
    #q.put((num, pos))
    #q.put((n - pos, (num + n - pos) % n))
    flag = flag and check_n_put((num, pos), q, row, free_nums)
    flag = flag and check_n_put((n - pos, (num + n - pos) % n), q, row, free_nums)
    #'''
    if (pos > 1 and num > 1):  #   . . (neighbour) (pos, num) . . . (brother, num - 1) (sister) . .
        brother_num = num - 1
        brother_pos = free_nums[brother_num]
        neighbour_pos = pos - 1
        neighbour_num = row[neighbour_pos]
        if (brother_pos != 0 and brother_pos < n - 1):
            sister_pos = brother_pos + 1
            sister_num = row[sister_pos]
            if (sister_num != 0):
                #q.put((neighbour_pos, sister_num - 1))
                flag = flag and check_n_put((neighbour_pos, sister_num - 1), q, row, free_nums)
            if (neighbour_num != 0):
                #q.put((sister_pos, neighbour_num + 1))
                flag = flag and check_n_put((sister_pos, neighbour_num + 1), q, row, free_nums)
        if (neighbour_num != 0):
            sister_num = neighbour_num + 1
            sister_pos = free_nums[sister_num]
            if (sister_pos != 0):
                brother_pos = sister_pos - 1
                #q.put((brother_pos, brother_num))
                flag = flag and check_n_put((brother_pos, brother_num), q, row, free_nums)
                
    if (pos < n - 1 and num < n - 1):  #   . . (pos, num) (neighbour) . . . (sister) (brother, num + 1) . .
        brother_num = num + 1
        brother_pos = free_nums[brother_num]
        neighbour_pos = pos + 1
        neighbour_num = row[neighbour_pos]
        if (brother_pos != 0 and brother_pos > 1):
            sister_pos = brother_pos - 1
            sister_num = row[sister_pos]
            if (sister_num != 0):
                #q.put((neighbour_pos, sister_num + 1))
                flag = flag and check_n_put((neighbour_pos, sister_num + 1), q, row, free_nums)
            if (neighbour_num != 0):
                #q.put((sister_pos, neighbour_num - 1))
                flag = flag and check_n_put((sister_pos, neighbour_num - 1), q, row, free_nums)
        if (neighbour_num != 0):
            sister_num = neighbour_num - 1
            sister_pos = free_nums[sister_num]
            if (sister_pos != 0):
                brother_pos = sister_pos + 1
                #q.put((brother_pos, brother_num))
                flag = flag and check_n_put((brother_pos, brother_num), q, row, free_nums)
    #'''
    
    res = flag and insert_recursive(n, p, m, row, free_nums, q, changes)
    
    if (not res):
        row[pos] = 0
        free_nums[num] = 0
        
    return res

In [203]:
def generate_rows(n, p, m):
    ans = []
    
    free_nums = [0 for i in range(n)] # zero = free, non-zero = position in a row
    free_nums[0] = n
    
    row = [0 for i in range(n)]

    Stopwatch.start()
    generate_row_recursive(n, p, m, row, free_nums, ans, 0)
    
    return ans

In [131]:
def find_all_tables(p, m):
    n = 1
    for i in range(m):
        n *= p
    n -= 1
    
    rows = generate_rows(n, p, m)
    #print('---------')
    for l in rows:
        print(l)
        print('---------')
        t = build_table(l)
        print_table(t)
        print('---------')
        print(check_table(t))

In [215]:
find_all_tables(2, 6)

progress: 6.34 %, time = 0.0009999275207519531
progress: 7.93 %, time = 0.1370079517364502
progress: 9.52 %, time = 10.90562391281128
[0, 6, 12, 32, 24, 62, 1, 26, 48, 45, 61, 25, 2, 35, 52, 23, 33, 47, 27, 56, 59, 42, 50, 15, 4, 11, 7, 18, 41, 60, 46, 34, 3, 16, 31, 13, 54, 44, 49, 43, 55, 28, 21, 39, 37, 9, 30, 17, 8, 38, 22, 53, 14, 51, 36, 40, 19, 58, 57, 20, 29, 10, 5]
progress: 11.11 %, time = 45.116172552108765
progress: 12.69 %, time = 198.81049227714539
[0, 8, 16, 53, 32, 38, 43, 62, 1, 45, 13, 51, 23, 10, 61, 44, 2, 41, 27, 34, 26, 42, 39, 12, 46, 30, 20, 18, 59, 48, 25, 35, 4, 58, 19, 31, 54, 57, 5, 22, 52, 17, 21, 6, 15, 9, 24, 49, 29, 47, 60, 11, 40, 3, 36, 56, 55, 37, 33, 28, 50, 14, 7]
progress: 14.28 %, time = 458.2932312488556
progress: 15.87 %, time = 646.1693108081818
progress: 17.46 %, time = 752.6676075458527
progress: 19.04 %, time = 829.1287486553192
progress: 20.63 %, time = 908.148886680603
progress: 22.22 %, time = 1006.7710571289062
progress: 23.8 %, time = 1

 22 50 43 61 59 31 52 39 30 60 44 12 36 10 58 62 41 17 16 42 51 32 27 -1 28 34 54 46 21 23 48  7  4 20 47 24 57 11 45 55  6 49 15 18  1  9 37 26 33 29 40  0 19  5 56 25 38 53 35 13  3  8  2 14
 23 15 51 44 62 60 32 53 40 31 61 45 13 37 11 59  0 42 18 17 43 52 33 28 -1 29 35 55 47 22 24 49  8  5 21 48 25 58 12 46 56  7 50 16 19  2 10 38 27 34 30 41  1 20  6 57 26 39 54 36 14  4  9  3
 24  4 16 52 45  0 61 33 54 41 32 62 46 14 38 12 60  1 43 19 18 44 53 34 29 -1 30 36 56 48 23 25 50  9  6 22 49 26 59 13 47 57  8 51 17 20  3 11 39 28 35 31 42  2 21  7 58 27 40 55 37 15  5 10
 25 11  5 17 53 46  1 62 34 55 42 33  0 47 15 39 13 61  2 44 20 19 45 54 35 30 -1 31 37 57 49 24 26 51 10  7 23 50 27 60 14 48 58  9 52 18 21  4 12 40 29 36 32 43  3 22  8 59 28 41 56 38 16  6
 26  7 12  6 18 54 47  2  0 35 56 43 34  1 48 16 40 14 62  3 45 21 20 46 55 36 31 -1 32 38 58 50 25 27 52 11  8 24 51 28 61 15 49 59 10 53 19 22  5 13 41 30 37 33 44  4 23  9 60 29 42 57 39 17
 27 18  8 13  7 19 55 48  3  1 36 5

  5 38 33 55 19 12 -1 13 21 58 37 43 48  4  6 50 18 56 28 15  3 49  7 46 32 39 31 47 44 17 51 35 25 23  1 53 30 40  9  0 24 36 59 62 10 27 57 22 26 11 20 14 29 54 34 52  2 16 45  8 41 61 60 42
  6 43 39 34 56 20 13 -1 14 22 59 38 44 49  5  7 51 19 57 29 16  4 50  8 47 33 40 32 48 45 18 52 36 26 24  2 54 31 41 10  1 25 37 60  0 11 28 58 23 27 12 21 15 30 55 35 53  3 17 46  9 42 62 61
  7 62 44 40 35 57 21 14 -1 15 23 60 39 45 50  6  8 52 20 58 30 17  5 51  9 48 34 41 33 49 46 19 53 37 27 25  3 55 32 42 11  2 26 38 61  1 12 29 59 24 28 13 22 16 31 56 36 54  4 18 47 10 43  0
  8  1  0 45 41 36 58 22 15 -1 16 24 61 40 46 51  7  9 53 21 59 31 18  6 52 10 49 35 42 34 50 47 20 54 38 28 26  4 56 33 43 12  3 27 39 62  2 13 30 60 25 29 14 23 17 32 57 37 55  5 19 48 11 44
  9 45  2  1 46 42 37 59 23 16 -1 17 25 62 41 47 52  8 10 54 22 60 32 19  7 53 11 50 36 43 35 51 48 21 55 39 29 27  5 57 34 44 13  4 28 40  0  3 14 31 61 26 30 15 24 18 33 58 38 56  6 20 49 12
 10 13 46  3  2 47 43 38 60 24 17 -

 51 11 61 49 32 53 29 15 22 14 30 27  0 34 18  8  6 47 36 13 23 55 46  7 19 42 45 56 10 40  5  9 57  3 60 12 37 17 35 48 62 28 54 24 44 43 25 21 16 38  2 58 -1 59  4 41 20 26 31 50 52 33  1 39
 52 40 12 62 50 33 54 30 16 23 15 31 28  1 35 19  9  7 48 37 14 24 56 47  8 20 43 46 57 11 41  6 10 58  4 61 13 38 18 36 49  0 29 55 25 45 44 26 22 17 39  3 59 -1 60  5 42 21 27 32 51 53 34  2
 53  3 41 13  0 51 34 55 31 17 24 16 32 29  2 36 20 10  8 49 38 15 25 57 48  9 21 44 47 58 12 42  7 11 59  5 62 14 39 19 37 50  1 30 56 26 46 45 27 23 18 40  4 60 -1 61  6 43 22 28 33 52 54 35
 54 36  4 42 14  1 52 35 56 32 18 25 17 33 30  3 37 21 11  9 50 39 16 26 58 49 10 22 45 48 59 13 43  8 12 60  6  0 15 40 20 38 51  2 31 57 27 47 46 28 24 19 41  5 61 -1 62  7 44 23 29 34 53 55
 55 56 37  5 43 15  2 53 36 57 33 19 26 18 34 31  4 38 22 12 10 51 40 17 27 59 50 11 23 46 49 60 14 44  9 13 61  7  1 16 41 21 39 52  3 32 58 28 48 47 29 25 20 42  6 62 -1  0  8 45 24 30 35 54
 56 55 57 38  6 44 16  3 54 37 58 3

 30 26 50 25 12 34  6  5 35 24 51 44 62  3 23 36 28 59 32 49 27 37 48 33 13  8  2  0 19 15 54 -1 55 17 22  4  7 14 20 41 57 47 38 61 45 10 43 52 40 21 18  1  9 46 58 29 31 60 39 53 16 56 42 11
 31 12 27 51 26 13 35  7  6 36 25 52 45  0  4 24 37 29 60 33 50 28 38 49 34 14  9  3  1 20 16 55 -1 56 18 23  5  8 15 21 42 58 48 39 62 46 11 44 53 41 22 19  2 10 47 59 30 32 61 40 54 17 57 43
 32 44 13 28 52 27 14 36  8  7 37 26 53 46  1  5 25 38 30 61 34 51 29 39 50 35 15 10  4  2 21 17 56 -1 57 19 24  6  9 16 22 43 59 49 40  0 47 12 45 54 42 23 20  3 11 48 60 31 33 62 41 55 18 58
 33 59 45 14 29 53 28 15 37  9  8 38 27 54 47  2  6 26 39 31 62 35 52 30 40 51 36 16 11  5  3 22 18 57 -1 58 20 25  7 10 17 23 44 60 50 41  1 48 13 46 55 43 24 21  4 12 49 61 32 34  0 42 56 19
 34 20 60 46 15 30 54 29 16 38 10  9 39 28 55 48  3  7 27 40 32  0 36 53 31 41 52 37 17 12  6  4 23 19 58 -1 59 21 26  8 11 18 24 45 61 51 42  2 49 14 47 56 44 25 22  5 13 50 62 33 35  1 43 57
 35 58 21 61 47 16 31 55 30 17 39 1

 13 61 45  5 59 49  2 23 29 36 39 21 26 51 -1 52 28 24 43 41 35 30 10 58  6 16 57 11 47 15  7 20 40 44 62 55 19  8 38 37  9 31 18 56 17 32  1 50 27 53  4 46 12 14 48 60 34 42 25 22  3 54  0 33
 14 34 62 46  6 60 50  3 24 30 37 40 22 27 52 -1 53 29 25 44 42 36 31 11 59  7 17 58 12 48 16  8 21 41 45  0 56 20  9 39 38 10 32 19 57 18 33  2 51 28 54  5 47 13 15 49 61 35 43 26 23  4 55  1
 15  2 35  0 47  7 61 51  4 25 31 38 41 23 28 53 -1 54 30 26 45 43 37 32 12 60  8 18 59 13 49 17  9 22 42 46  1 57 21 10 40 39 11 33 20 58 19 34  3 52 29 55  6 48 14 16 50 62 36 44 27 24  5 56
 16 57  3 36  1 48  8 62 52  5 26 32 39 42 24 29 54 -1 55 31 27 46 44 38 33 13 61  9 19 60 14 50 18 10 23 43 47  2 58 22 11 41 40 12 34 21 59 20 35  4 53 30 56  7 49 15 17 51  0 37 45 28 25  6
 17  7 58  4 37  2 49  9  0 53  6 27 33 40 43 25 30 55 -1 56 32 28 47 45 39 34 14 62 10 20 61 15 51 19 11 24 44 48  3 59 23 12 42 41 13 35 22 60 21 36  5 54 31 57  8 50 16 18 52  1 38 46 29 26
 18 27  8 59  5 38  3 50 10  1 54  

 58 23 17 12 55 40 51 61 39 56 29 60 52  2 22 26 44 37  1 53 20 19 54 13  0 38 62 14 46 32  9 35 49 28 57 59 30 42 16 24  7  4 48 36 45 15 43 27 50 41 31 47  5 11 18 21  3  8 33 -1 34 10  6 25
 59 26 24 18 13 56 41 52 62 40 57 30 61 53  3 23 27 45 38  2 54 21 20 55 14  1 39  0 15 47 33 10 36 50 29 58 60 31 43 17 25  8  5 49 37 46 16 44 28 51 42 32 48  6 12 19 22  4  9 34 -1 35 11  7
 60  8 27 25 19 14 57 42 53  0 41 58 31 62 54  4 24 28 46 39  3 55 22 21 56 15  2 40  1 16 48 34 11 37 51 30 59 61 32 44 18 26  9  6 50 38 47 17 45 29 52 43 33 49  7 13 20 23  5 10 35 -1 36 12
 61 13  9 28 26 20 15 58 43 54  1 42 59 32  0 55  5 25 29 47 40  4 56 23 22 57 16  3 41  2 17 49 35 12 38 52 31 60 62 33 45 19 27 10  7 51 39 48 18 46 30 53 44 34 50  8 14 21 24  6 11 36 -1 37
 62 38 14 10 29 27 21 16 59 44 55  2 43 60 33  1 56  6 26 30 48 41  5 57 24 23 58 17  4 42  3 18 50 36 13 39 53 32 61  0 34 46 20 28 11  8 52 40 49 19 47 31 54 45 35 51  9 15 22 25  7 12 37 -1
---------
True
[0, 56, 49, 13, 35, 

 38 33 44 47  7 19 43 34  3 13 53 42 20 18  8 55 26 62 59 12  4 11 60 36 57 40 28 15 50 25 56 37 39 58  0  6 48 22 30 -1 31 24 51 10  5  1 46 45  2 35 61 27 41 54  9 52 14 29 23 32 17 21 49 16
 39 17 34 45 48  8 20 44 35  4 14 54 43 21 19  9 56 27  0 60 13  5 12 61 37 58 41 29 16 51 26 57 38 40 59  1  7 49 23 31 -1 32 25 52 11  6  2 47 46  3 36 62 28 42 55 10 53 15 30 24 33 18 22 50
 40 51 18 35 46 49  9 21 45 36  5 15 55 44 22 20 10 57 28  1 61 14  6 13 62 38 59 42 30 17 52 27 58 39 41 60  2  8 50 24 32 -1 33 26 53 12  7  3 48 47  4 37  0 29 43 56 11 54 16 31 25 34 19 23
 41 24 52 19 36 47 50 10 22 46 37  6 16 56 45 23 21 11 58 29  2 62 15  7 14  0 39 60 43 31 18 53 28 59 40 42 61  3  9 51 25 33 -1 34 27 54 13  8  4 49 48  5 38  1 30 44 57 12 55 17 32 26 35 20
 42 21 25 53 20 37 48 51 11 23 47 38  7 17 57 46 24 22 12 59 30  3  0 16  8 15  1 40 61 44 32 19 54 29 60 41 43 62  4 10 52 26 34 -1 35 28 55 14  9  5 50 49  6 39  2 31 45 58 13 56 18 33 27 36
 43 37 22 26 54 21 38 49 52 12 24 4

 20 24 27 56 36 50 60 31 48 18 58 22 38 35 57 19 21 59 51  8 14 -1 15 10 54  0 26 25  1 43 47 32  6 30 61 45 12  3 53 11 46 44 62 55 28 40 34 39 29  7 52  4 17 49 37 23 42  2 13  9 16  5 33 41
 21 42 25 28 57 37 51 61 32 49 19 59 23 39 36 58 20 22 60 52  9 15 -1 16 11 55  1 27 26  2 44 48 33  7 31 62 46 13  4 54 12 47 45  0 56 29 41 35 40 30  8 53  5 18 50 38 24 43  3 14 10 17  6 34
 22 35 43 26 29 58 38 52 62 33 50 20 60 24 40 37 59 21 23 61 53 10 16 -1 17 12 56  2 28 27  3 45 49 34  8 32  0 47 14  5 55 13 48 46  1 57 30 42 36 41 31  9 54  6 19 51 39 25 44  4 15 11 18  7
 23  8 36 44 27 30 59 39 53  0 34 51 21 61 25 41 38 60 22 24 62 54 11 17 -1 18 13 57  3 29 28  4 46 50 35  9 33  1 48 15  6 56 14 49 47  2 58 31 43 37 42 32 10 55  7 20 52 40 26 45  5 16 12 19
 24 20  9 37 45 28 31 60 40 54  1 35 52 22 62 26 42 39 61 23 25  0 55 12 18 -1 19 14 58  4 30 29  5 47 51 36 10 34  2 49 16  7 57 15 50 48  3 59 32 44 38 43 33 11 56  8 21 53 41 27 46  6 17 13
 25 14 21 10 38 46 29 32 61 41 55  

In [214]:
find_all_tables(2, 5)

progress: 12.9 %, time = 0.0009999275207519531
progress: 16.12 %, time = 0.01500082015991211
progress: 19.35 %, time = 0.13200736045837402
progress: 22.58 %, time = 0.24301385879516602
progress: 25.8 %, time = 0.3060173988342285
progress: 29.03 %, time = 0.365020751953125
progress: 32.25 %, time = 0.4190237522125244
progress: 35.48 %, time = 0.489027738571167
progress: 38.7 %, time = 0.5870335102081299
[0, 12, 24, 8, 17, 28, 16, 9, 3, 7, 25, 30, 1, 27, 18, 21, 6, 4, 14, 20, 19, 15, 29, 26, 2, 10, 23, 13, 5, 22, 11]
progress: 41.93 %, time = 0.6470367908477783
[0, 13, 26, 23, 21, 7, 15, 5, 11, 25, 14, 8, 30, 1, 10, 6, 22, 27, 19, 18, 28, 4, 16, 3, 29, 9, 2, 17, 20, 24, 12]
progress: 45.16 %, time = 0.722041130065918
[0, 14, 28, 5, 25, 3, 10, 16, 19, 24, 6, 23, 20, 30, 1, 22, 7, 18, 17, 8, 12, 27, 15, 11, 9, 4, 29, 21, 2, 26, 13]
progress: 58.06 %, time = 0.8220467567443848
[0, 18, 5, 29, 10, 2, 27, 22, 20, 16, 4, 19, 23, 14, 13, 24, 9, 30, 1, 11, 8, 25, 7, 12, 15, 21, 28, 6, 26, 3, 17]


  1 14 -1 15 29  6 26  4 11 17 20 25  7 24 21  0  2 23  8 19 18  9 13 28 16 12 10  5 30 22  3 27
  2 28 15 -1 16 30  7 27  5 12 18 21 26  8 25 22  1  3 24  9 20 19 10 14 29 17 13 11  6  0 23  4
  3  5 29 16 -1 17  0  8 28  6 13 19 22 27  9 26 23  2  4 25 10 21 20 11 15 30 18 14 12  7  1 24
  4 25  6 30 17 -1 18  1  9 29  7 14 20 23 28 10 27 24  3  5 26 11 22 21 12 16  0 19 15 13  8  2
  5  3 26  7  0 18 -1 19  2 10 30  8 15 21 24 29 11 28 25  4  6 27 12 23 22 13 17  1 20 16 14  9
  6 10  4 27  8  1 19 -1 20  3 11  0  9 16 22 25 30 12 29 26  5  7 28 13 24 23 14 18  2 21 17 15
  7 16 11  5 28  9  2 20 -1 21  4 12  1 10 17 23 26  0 13 30 27  6  8 29 14 25 24 15 19  3 22 18
  8 19 17 12  6 29 10  3 21 -1 22  5 13  2 11 18 24 27  1 14  0 28  7  9 30 15 26 25 16 20  4 23
  9 24 20 18 13  7 30 11  4 22 -1 23  6 14  3 12 19 25 28  2 15  1 29  8 10  0 16 27 26 17 21  5
 10  6 25 21 19 14  8  0 12  5 23 -1 24  7 15  4 13 20 26 29  3 16  2 30  9 11  1 17 28 27 18 22
 11 23  7 26 22 20 15  9  1 13

 30 18  6 10 13 28 21  1 27 14 26  2 12 11  3  8 24 20 29  0 22 16  5 19 25 15 23  9  7  4 17 -1
---------
True
[0, 20, 9, 26, 18, 8, 21, 29, 5, 2, 16, 12, 11, 17, 27, 25, 10, 13, 4, 30, 1, 6, 24, 28, 22, 15, 3, 14, 23, 7, 19]
---------
 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
  0 -1 20  9 26 18  8 21 29  5  2 16 12 11 17 27 25 10 13  4 30  1  6 24 28 22 15  3 14 23  7 19
  1 20 -1 21 10 27 19  9 22 30  6  3 17 13 12 18 28 26 11 14  5  0  2  7 25 29 23 16  4 15 24  8
  2  9 21 -1 22 11 28 20 10 23  0  7  4 18 14 13 19 29 27 12 15  6  1  3  8 26 30 24 17  5 16 25
  3 26 10 22 -1 23 12 29 21 11 24  1  8  5 19 15 14 20 30 28 13 16  7  2  4  9 27  0 25 18  6 17
  4 18 27 11 23 -1 24 13 30 22 12 25  2  9  6 20 16 15 21  0 29 14 17  8  3  5 10 28  1 26 19  7
  5  8 19 28 12 24 -1 25 14  0 23 13 26  3 10  7 21 17 16 22  1 30 15 18  9  4  6 11 29  2 27 20
  6 21  9 20 29 13 25 -1 26 15  1 24 14 27  4 11  8 22 18 17 23  2  0 16 19 10  5  7

In [213]:
find_all_tables(2, 4)

progress: 26.66 %, time = 0.0020003318786621094
[0, 4, 8, 14, 1, 10, 13, 9, 2, 7, 5, 12, 11, 6, 3]
progress: 33.33 %, time = 0.010000467300415039
progress: 40.0 %, time = 0.023001432418823242
progress: 66.66 %, time = 0.029001712799072266
progress: 73.33 %, time = 0.03300189971923828
progress: 80.0 %, time = 0.0370020866394043
[0, 12, 9, 4, 3, 10, 8, 13, 6, 2, 5, 14, 1, 7, 11]
[0, 4, 8, 14, 1, 10, 13, 9, 2, 7, 5, 12, 11, 6, 3]
---------
 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
  0 -1  4  8 14  1 10 13  9  2  7  5 12 11  6  3
  1  4 -1  5  9  0  2 11 14 10  3  8  6 13 12  7
  2  8  5 -1  6 10  1  3 12  0 11  4  9  7 14 13
  3 14  9  6 -1  7 11  2  4 13  1 12  5 10  8  0
  4  1  0 10  7 -1  8 12  3  5 14  2 13  6 11  9
  5 10  2  1 11  8 -1  9 13  4  6  0  3 14  7 12
  6 13 11  3  2 12  9 -1 10 14  5  7  1  4  0  8
  7  9 14 12  4  3 13 10 -1 11  0  6  8  2  5  1
  8  2 10  0 13  5  4 14 11 -1 12  1  7  9  3  6
  9  7  3 11  1 14  6  5  0 12 -1 13  2  8 10  4
 10  5  8  4 12  2  

In [212]:
find_all_tables(2, 3)

progress: 42.85 %, time = 0.0009999275207519531
[0, 3, 6, 1, 5, 4, 2]
progress: 71.42 %, time = 0.0019998550415039062
[0, 5, 3, 2, 6, 1, 4]
[0, 3, 6, 1, 5, 4, 2]
---------
 -1  0  1  2  3  4  5  6
  0 -1  3  6  1  5  4  2
  1  3 -1  4  0  2  6  5
  2  6  4 -1  5  1  3  0
  3  1  0  5 -1  6  2  4
  4  5  2  1  6 -1  0  3
  5  4  6  3  2  0 -1  1
  6  2  5  0  4  3  1 -1
---------
True
[0, 5, 3, 2, 6, 1, 4]
---------
 -1  0  1  2  3  4  5  6
  0 -1  5  3  2  6  1  4
  1  5 -1  6  4  3  0  2
  2  3  6 -1  0  5  4  1
  3  2  4  0 -1  1  6  5
  4  6  3  5  1 -1  2  0
  5  1  0  4  6  2 -1  3
  6  4  2  1  5  0  3 -1
---------
True


In [211]:
find_all_tables(2, 2)

progress: 66.66 %, time = 0.0010001659393310547
[0, 2, 1]
[0, 2, 1]
---------
 -1  0  1  2
  0 -1  2  1
  1  2 -1  0
  2  1  0 -1
---------
True


In [210]:
find_all_tables(2, 1)

[0]
[0]
---------
 -1  0
  0 -1
---------
True


In [77]:
test_props(2, 6)
# FAILED
#1. distributivity
#2. associativity between rows x and 0
#3. associativity between rows 0 and 1

KeyboardInterrupt: 

In [76]:
test_props(2, 5)
#1. distributivity
#2. associativity between rows x and 0
#3. associativity between rows 0 and 1

[0, 12, 24, 8, 17, 28, 16, 9, 3, 7, 25, 30, 1, 27, 18, 21, 6, 4, 14, 20, 19, 15, 29, 26, 2, 10, 23, 13, 5, 22, 11]
---------
 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
  0 -1 12 24  8 17 28 16  9  3  7 25 30  1 27 18 21  6  4 14 20 19 15 29 26  2 10 23 13  5 22 11
  1 12 -1 13 25  9 18 29 17 10  4  8 26  0  2 28 19 22  7  5 15 21 20 16 30 27  3 11 24 14  6 23
  2 24 13 -1 14 26 10 19 30 18 11  5  9 27  1  3 29 20 23  8  6 16 22 21 17  0 28  4 12 25 15  7
  3  8 25 14 -1 15 27 11 20  0 19 12  6 10 28  2  4 30 21 24  9  7 17 23 22 18  1 29  5 13 26 16
  4 17  9 26 15 -1 16 28 12 21  1 20 13  7 11 29  3  5  0 22 25 10  8 18 24 23 19  2 30  6 14 27
  5 28 18 10 27 16 -1 17 29 13 22  2 21 14  8 12 30  4  6  1 23 26 11  9 19 25 24 20  3  0  7 15
  6 16 29 19 11 28 17 -1 18 30 14 23  3 22 15  9 13  0  5  7  2 24 27 12 10 20 26 25 21  4  1  8
  7  9 17 30 20 12 29 18 -1 19  0 15 24  4 23 16 10 14  1  6  8  3 25 28 13 11 21 27 26 22  5  2
  

 25  4 10 13 18  0 17 14 24 26 16  1 12 11  2  6 21  9  5  3 29 23 15 27 20  7 -1  8 22 30 19 28
 26 29  5 11 14 19  1 18 15 25 27 17  2 13 12  3  7 22 10  6  4 30 24 16 28 21  8 -1  9 23  0 20
 27 21 30  6 12 15 20  2 19 16 26 28 18  3 14 13  4  8 23 11  7  5  0 25 17 29 22  9 -1 10 24  1
 28  2 22  0  7 13 16 21  3 20 17 27 29 19  4 15 14  5  9 24 12  8  6  1 26 18 30 23 10 -1 11 25
 29 26  3 23  1  8 14 17 22  4 21 18 28 30 20  5 16 15  6 10 25 13  9  7  2 27 19  0 24 11 -1 12
 30 13 27  4 24  2  9 15 18 23  5 22 19 29  0 21  6 17 16  7 11 26 14 10  8  3 28 20  1 25 12 -1
---------
True
[0, 18, 5, 29, 10, 2, 27, 22, 20, 16, 4, 19, 23, 14, 13, 24, 9, 30, 1, 11, 8, 25, 7, 12, 15, 21, 28, 6, 26, 3, 17]
---------
 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
  0 -1 18  5 29 10  2 27 22 20 16  4 19 23 14 13 24  9 30  1 11  8 25  7 12 15 21 28  6 26  3 17
  1 18 -1 19  6 30 11  3 28 23 21 17  5 20 24 15 14 25 10  0  2 12  9 26  8 13 16 22

In [121]:
test_props(2, 4)
#1. distributivity
#2. associativity between rows x and 0
#3. associativity between rows 0 and 1

[0, 4, 8, 14, 1, 10, 13, 9, 2, 7, 5, 12, 11, 6, 3]
---------
 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
  0 -1  4  8 14  1 10 13  9  2  7  5 12 11  6  3
  1  4 -1  5  9  0  2 11 14 10  3  8  6 13 12  7
  2  8  5 -1  6 10  1  3 12  0 11  4  9  7 14 13
  3 14  9  6 -1  7 11  2  4 13  1 12  5 10  8  0
  4  1  0 10  7 -1  8 12  3  5 14  2 13  6 11  9
  5 10  2  1 11  8 -1  9 13  4  6  0  3 14  7 12
  6 13 11  3  2 12  9 -1 10 14  5  7  1  4  0  8
  7  9 14 12  4  3 13 10 -1 11  0  6  8  2  5  1
  8  2 10  0 13  5  4 14 11 -1 12  1  7  9  3  6
  9  7  3 11  1 14  6  5  0 12 -1 13  2  8 10  4
 10  5  8  4 12  2  0  7  6  1 13 -1 14  3  9 11
 11 12  6  9  5 13  3  1  8  7  2 14 -1  0  4 10
 12 11 13  7 10  6 14  4  2  9  8  3  0 -1  1  5
 13  6 12 14  8 11  7  0  5  3 10  9  4  1 -1  2
 14  3  7 13  0  9 12  8  1  6  4 11 10  5  2 -1
---------
True
[0, 12, 9, 4, 3, 10, 8, 13, 6, 2, 5, 14, 1, 7, 11]
---------
 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
  0 -1 12  9  4  3 10  8 13  6

In [122]:
test_props(2, 3)
#1. distributivity
#2. associativity between rows x and 0
#3. associativity between rows 0 and 1

[0, 3, 6, 1, 5, 4, 2]
---------
 -1  0  1  2  3  4  5  6
  0 -1  3  6  1  5  4  2
  1  3 -1  4  0  2  6  5
  2  6  4 -1  5  1  3  0
  3  1  0  5 -1  6  2  4
  4  5  2  1  6 -1  0  3
  5  4  6  3  2  0 -1  1
  6  2  5  0  4  3  1 -1
---------
True
[0, 5, 3, 2, 6, 1, 4]
---------
 -1  0  1  2  3  4  5  6
  0 -1  5  3  2  6  1  4
  1  5 -1  6  4  3  0  2
  2  3  6 -1  0  5  4  1
  3  2  4  0 -1  1  6  5
  4  6  3  5  1 -1  2  0
  5  1  0  4  6  2 -1  3
  6  4  2  1  5  0  3 -1
---------
True


In [125]:
test_props(2, 2)
#1. distributivity
#2. associativity between rows x and 0
#3. associativity between rows 0 and 1

[0, 2, 1]
---------
 -1  0  1  2
  0 -1  2  1
  1  2 -1  0
  2  1  0 -1
---------
True


In [123]:
test_props(2, 1)
#1. distributivity
#2. associativity between rows x and 0
#3. associativity between rows 0 and 1

[0]
---------
 -1  0
  0 -1
---------
True


In [11]:
test(2, 5)
#1. distributivity
#2. associativity between rows x and 0

[[0, 3, 30, 1, 11, 15, 18, 27, 17, 23, 26, 4, 25, 19, 22, 5, 21, 8, 6, 13, 24, 16, 14, 9, 20, 12, 10, 7, 29, 28, 2], [0, 3, 30, 1, 11, 15, 18, 27, 22, 17, 26, 4, 25, 19, 23, 5, 21, 9, 6, 13, 24, 16, 8, 14, 20, 12, 10, 7, 29, 28, 2], [0, 3, 30, 1, 11, 15, 19, 27, 17, 23, 26, 4, 18, 25, 22, 5, 21, 8, 12, 6, 24, 16, 14, 9, 20, 13, 10, 7, 29, 28, 2], [0, 3, 30, 1, 11, 15, 19, 27, 22, 17, 26, 4, 18, 25, 23, 5, 21, 9, 12, 6, 24, 16, 8, 14, 20, 13, 10, 7, 29, 28, 2], [0, 3, 30, 1, 11, 17, 15, 27, 18, 25, 23, 4, 26, 21, 19, 6, 22, 5, 8, 14, 24, 13, 16, 10, 20, 9, 12, 7, 29, 28, 2], [0, 3, 30, 1, 11, 17, 15, 27, 21, 25, 18, 4, 26, 23, 19, 6, 22, 5, 10, 14, 24, 8, 16, 13, 20, 9, 12, 7, 29, 28, 2], [0, 3, 30, 1, 11, 17, 22, 27, 18, 15, 23, 4, 26, 21, 19, 9, 25, 5, 8, 14, 24, 13, 6, 10, 20, 16, 12, 7, 29, 28, 2], [0, 3, 30, 1, 11, 17, 22, 27, 21, 15, 18, 4, 26, 23, 19, 9, 25, 5, 10, 14, 24, 8, 6, 13, 20, 16, 12, 7, 29, 28, 2], [0, 3, 30, 1, 11, 19, 15, 27, 18, 25, 23, 4, 17, 21, 26, 6, 22, 12, 8, 

In [12]:
test(2, 4)
#1. distributivity
#2. associativity between rows x and 0

[[0, 4, 8, 14, 1, 10, 13, 9, 2, 7, 5, 12, 11, 6, 3], [0, 4, 9, 14, 1, 10, 8, 13, 6, 2, 5, 12, 11, 7, 3], [0, 12, 8, 4, 3, 10, 13, 9, 2, 7, 5, 14, 1, 6, 11], [0, 12, 9, 4, 3, 10, 8, 13, 6, 2, 5, 14, 1, 7, 11]]


In [13]:
test(2, 3)
#1. distributivity
#2. associativity between rows x and 0

[[0, 3, 6, 1, 5, 4, 2], [0, 5, 3, 2, 6, 1, 4]]


In [116]:
test(2, 2)
#1. distributivity
#2. associativity between rows x and 0

[[0, 2, 1]]


In [117]:
test(2, 1)
#1. distributivity
#2. associativity between rows x and 0

[[0]]
